<a href="https://colab.research.google.com/github/Karthick47v2/mcq-generator/blob/main/summarize-extract-genQ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# TODO : OPTIMIZEEEEEE

In [ ]:
# DONT FORGET TO CHANGE TO CUDA VERSION
# !pip3 install torch==1.8.2+cpu torchvision==0.9.2+cpu torchaudio==0.8.2 -f https://download.pytorch.org/whl/lts/1.8/torch_lts.html
!pip3 install transformers==4.15.0    # only this ver works with FastT5
!pip3 install SentencePiece
!pip3 install git+https://github.com/boudinfl/pke.git
# FAST T5
!pip3 install fastt5

In [2]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
import pke
import string

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [9]:
from fastT5 import export_and_get_onnx_model, generate_onnx_representation, quantize
from transformers import T5Config, AutoTokenizer

In [10]:
# m = 'ramsrigouthamg/t5_squad_v1'
# onnx_model_paths = generate_onnx_representation(m)


Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

Exporting to onnx... |################################| 3/3


In [11]:
# quant_model_paths = quantize(onnx_model_paths)

# tokenizer_onnx = AutoTokenizer.from_pretrained(m)
# config = T5Config.from_pretrained(m)

Quantizing... |################################| 3/3


Downloading:   0%|          | 0.00/1.81k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.74k [00:00<?, ?B/s]

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [12]:
# tokenizer_onnx.save_pretrained('models/')
# config.save_pretrained('models/')


In [13]:
# !cp -r models '/content/gdrive/MyDrive/mcq-gen/'

In [14]:
from fastT5 import get_onnx_model, get_onnx_runtime_sessions, OnnxT5 
from transformers import AutoTokenizer
from pathlib import Path
import os

# summarize
model_path = '/content/gdrive/MyDrive/mcq-gen/t5-summarize'
model_name = "t5-large"
encoder_path = os.path.join(model_path, f"{model_name}-encoder-quantized.onnx")
decoder_path = os.path.join(model_path, f"{model_name}-decoder-quantized.onnx")
init_decoder_path = os.path.join(model_path, f"{model_name}-init-decoder-quantized.onnx")

model_sessions = get_onnx_runtime_sessions((encoder_path,decoder_path,init_decoder_path))
sum_model = OnnxT5(model_path, model_sessions)
sum_tokenizer = AutoTokenizer.from_pretrained(model_path)

# que model
model_path = '/content/gdrive/MyDrive/mcq-gen/t5-question'
model_name = 't5_squad_v1'
encoder_path = os.path.join(model_path, f"{model_name}-encoder-quantized.onnx")
decoder_path = os.path.join(model_path, f"{model_name}-decoder-quantized.onnx")
init_decoder_path = os.path.join(model_path, f"{model_name}-init-decoder-quantized.onnx")

model_sessions = get_onnx_runtime_sessions((encoder_path,decoder_path,init_decoder_path))
q_model = OnnxT5(model_path, model_sessions)
q_tokenizer = AutoTokenizer.from_pretrained(model_path)

In [15]:
def preprocess_bulk_text(text):
  text = text.strip().replace("\n"," ")
  # https://huggingface.co/docs/transformers/preprocessing  - Tokenize
  encode = sum_tokenizer.encode_plus("summarize: " + text, return_tensors='pt', max_length=512, pad_to_max_length=False, truncation=True)
  return encode["input_ids"], encode["attention_mask"]

In [16]:
def postprocess_summary(text):
  output = ""

  for x in sent_tokenize(text):
    x = x.capitalize()
    output += " " + x

  return output

In [17]:
def summarize(text):
  input_tokens_ids, attention_mask = preprocess_bulk_text(text)
  summary_encoded = sum_model.generate(input_ids=input_tokens_ids, 
                                   attention_mask=attention_mask,
                                   num_beams=3,
                                   num_return_sequences=1,
                                   no_repeat_ngram_size=2, 
                                   max_length=300, 
                                   early_stopping=True)
  # decode summarized token
  output = sum_tokenizer.decode(summary_encoded[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)

  return postprocess_summary(output)

In [18]:
def extract_keywords(text, kw_pop):
  ex = pke.unsupervised.MultipartiteRank()
  ex.load_document(text)   

  pos = {'PROPN','NOUN'}
  stoplist = list(string.punctuation) 
  stoplist += ['-lrb-', '-rrb-', '-lcb-', '-rcb-', '-lsb-', '-rsb-']
  stoplist += stopwords.words('english')
  ex.candidate_selection(pos=pos, stoplist=stoplist)

  ex.candidate_weighting(alpha=1.1, threshold=0.75, method='average')

  kw = ex.get_n_best(n=kw_pop)

  kw_ls = []

  for i in kw:
    kw_ls.append(i[0])

  return kw_ls

In [19]:
def filter_keywords(original, summarized, kw_pop=10):
  orig_ls = extract_keywords(original, kw_pop)
  sum_ls = extract_keywords(summarized, kw_pop)

  orig_ls = set(orig_ls)
  return list(orig_ls.intersection(sum_ls))

In [20]:
full_text = """
Elon Musk has shown again he can influence the digital currency market with just his tweets. After saying that his electric vehicle-making company
Tesla will not accept payments in Bitcoin because of environmental concerns, he tweeted that he was working with developers of Dogecoin to improve
system transaction efficiency. Following the two distinct statements from him, the world's largest cryptocurrency hit a two-month low, while Dogecoin
rallied by about 20 percent. The SpaceX CEO has in recent months often tweeted in support of Dogecoin, but rarely for Bitcoin.  In a recent tweet,
Musk put out a statement from Tesla that it was “concerned” about the rapidly increasing use of fossil fuels for Bitcoin (price in India) mining and
transaction, and hence was suspending vehicle purchases using the cryptocurrency.  A day later he again tweeted saying, “To be clear, I strongly
believe in crypto, but it can't drive a massive increase in fossil fuel use, especially coal”.  It triggered a downward spiral for Bitcoin value but
the cryptocurrency has stabilised since.   A number of Twitter users welcomed Musk's statement. One of them said it's time people started realising
that Dogecoin “is here to stay” and another referred to Musk's previous assertion that crypto could become the world's future currency.
"""

summary = summarize(full_text)
print(summary)

 Elon musk has shown again he can influence the digital currency market with just his tweets. After saying that his electric vehicle-making company tesla will not accept payments in bitcoin because of environmental concerns, elon tweeted about dogecoin to improve system transaction efficiency. Following the two distinct statements from him, the world's largest cryptocurrency hit a two-month low.


In [21]:
filtered_kw = filter_keywords(full_text, summary)

In [22]:
filtered_kw

['elon musk', 'vehicle', 'tweets', 'bitcoin', 'dogecoin']

In [49]:
# T5_Q_MODEL_PATH = 'ramsrigouthamg/t5_squad_v1'
# question_model = T5ForConditionalGeneration.from_pretrained(T5_Q_MODEL_PATH)
# question_tokenizer = T5Tokenizer.from_pretrained(T5_Q_MODEL_PATH)

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.81k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.74k [00:00<?, ?B/s]

In [23]:
def preprocess_summary(context, answer):
  text = "context: {} answer: {}".format(context, answer)
  # https://huggingface.co/docs/transformers/preprocessing  - Tokenize
  encode = q_tokenizer.encode_plus(text, return_tensors='pt', max_length = 382, pad_to_max_length=False, truncation=True)
  return encode["input_ids"], encode["attention_mask"]

In [24]:
def gen_question(context, answer):
  input_tokens_ids, attention_mask = preprocess_summary(context, answer)

  question_encoded = q_model.generate(input_ids=input_tokens_ids, 
                                             attention_mask=attention_mask,
                                             num_beams=5,
                                             no_repeat_ngram_size=2, 
                                             max_length=72, 
                                             early_stopping=True)
  # decode summarized token
  output = q_tokenizer.decode(question_encoded[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
  output = output.replace("question: ", "")
  output = output.strip()
  return output

In [25]:
for kw in filtered_kw:
  print(kw)
  print(gen_question(summary, kw))

elon musk
Who has shown he can influence the digital currency market with just his tweets?
vehicle
What does tesla make?
tweets
What did elon musk use to influence the digital currency market?
bitcoin
What did elon musk say tesla would not accept payments in?
dogecoin
What did elon musk tweet about?
